## Sieć dla agenta do gry w Connect4 

In [197]:
import matplotlib.pyplot as plt
%matplotlib inline

import torch 
import torch.nn as nn
import torch.nn.functional as F

from torchsummary import summary

from imp import reload

import DataLoader
reload(DataLoader)

from DataLoader import InMemDataLoader
from DataLoader import C4DataSet

In [198]:
nr_of_games = 2
moves_observed  = 5
dataset = C4DataSet(nr_of_games, moves_observed).create_data_set(task_nr=2)

In [199]:
batch_size = 128
train_size, val_size, test_size = int(len(dataset)/3), int(len(dataset)/3), len(dataset)/3
amount_of_train_batches = train_size / batch_size

print(f"train_size = {train_size}")
train_set = dataset[:train_size]
val_set = dataset[train_size:train_size+val_size] 
test_set = dataset[train_size+val_size:]

data_loaders = {
    "train": InMemDataLoader(train_set, batch_size=batch_size, shuffle=True),
    "valid": InMemDataLoader(val_set, batch_size=batch_size, shuffle=False),
    "test": InMemDataLoader(test_set, batch_size=batch_size, shuffle=False),
}

train_size = 3


100%|██████████| 4/4 [00:00<00:00, 4425.54it/s]


In [200]:
import DataLoader
reload(DataLoader)
from DataLoader import C4DataSet

# game = 'S233322625223356466550404A' #<-- dobry przykłąd skosu, dziury 
# game = "S3256323326066002103304101B"
# game = "S3410241441125533221302A"
# dataset = C4DataSet(1, "all")
# samples = dataset.test_samples(game)
# for sample in samples:
#     print(sample)

print(train_set)

[(tensor([0., 2., 1., 0., 2., 0., 2., 0.]), 0), (tensor([0., 1., 1., 0., 1., 0., 2., 0.]), 0), (tensor([0., 2., 1., 0., 1., 0., 2., 0.]), 0)]


In [14]:
import wandb
wandb.login() # klucz - 7242fe50822869937a282970b5385963778c7f8c

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: michalfica125 (neuralnetworks). Use `wandb login --relogin` to force relogin


True

In [201]:
class Model(nn.Module):
    def __init__(self, dp=0.5):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(8, 2) # zamiast 2 wstawić 25 

        # self.bn3 = nn.BatchNorm1d(25)
        # self.fc2 = nn.Linear(25,2)
        # self.do = nn.Dropout(0.35)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        # x = self.do(x) 

        # x = self.fc2(x)
        # x = self.do(x)

        # x = nn.Softmax()(x)
        return x

    def loss(self, Out, Targets):
      return F.cross_entropy(Out, Targets)

model = Model()
summary(model, (1,8))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 2]              18
Total params: 18
Trainable params: 18
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [202]:
def compute_error_rate(model, data_loader, device="cpu"):
  model.eval()

  num_errs, num_examples = 0, 0
  with torch.no_grad():
    for batch in data_loader:

      x, y = batch[0].to(device), batch[1].to(device)
      out = model(x)

      _, pred = out.max(dim=1)
      num_errs += (pred != y.data).sum().item()
      num_examples += x.size(0)
    
  print(f"parametry sieci: ")
  for _, p in model.named_parameters():
    print(p)

  print(f"przy tych paramaetrach liczba pomyłek = {num_errs}")

  return num_errs / num_examples

In [242]:
def train(num_of_epochs, train_loader, opt, print_every=10, device="cpu"):
  model.train()

  for data_loader in data_loaders.values():
    if isinstance(data_loader, InMemDataLoader):
        data_loader.to(device)

  iter = 0
  for e in range(num_of_epochs):
    model.train()
    print(f"Epoch {e+1}")

    for batch in train_loader:
      iter += 1
      x = batch[0].to(device)
      y = batch[1].to(device)
      opt.zero_grad()
      
      out = model(x)
      # loss = nn.CrossEntropyLoss()(out, y)

      target = torch.randn(3, 2)
      loss = nn.MSELoss()(out, target)
      loss.backward()

      print(f"policzone gradienty: ")
      for _, p in model.named_parameters():
        print(p.grad)

      print(f"\nparametry PRZED sieci: ")
      for p in model.parameters():
        print(p.clone())
        print(p.requires_grad)

      opt.step()

      print(f"\nparametry PO sieci: ")
      for p in model.parameters():
        print(p.clone())
        print(p.requires_grad)

      _, pred = out.max(dim=1)
      batch_err = (pred != y).sum().item() / out.size(0)

      if iter % print_every == 0:
        print(f"iter = {iter}, batch_err = {batch_err * 100.0}")
        # wandb.log({"batch_error_rate": batch_err})

    print("\n\n")
    # val_err = compute_error_rate(model, data_loader=data_loaders["valid"], device=device)
    # print(f"val err = {100*val_err:.2f}")
    # wandb.log({'val_err': val_err, 'epoch': e+1})

In [115]:
def initialize_weights(model):
    with torch.no_grad():
        for name, p in model.named_parameters():
            if 'weight' in name:
                if 'conv' in name:
                    f_in = p.shape[1]*p.shape[2]*p.shape[3]
                    p.normal_(0, torch.sqrt(torch.tensor(2./f_in)))
                elif 'bn' in name:
                    p = torch.ones_like(p)
                elif 'fc' in name:
                    f_in = p.shape[1]
                    p.normal_(0, torch.sqrt(torch.tensor(2./f_in)))
                else:
                    raise Exception('weird weight')

            elif 'bias' in name:
                p.zero_()
            else:
                raise Exception('weird parameter')

In [116]:
cnt = 0

In [235]:
lr = 101
weight_decay = 0.0000001
momentum = 0.9
epochs = 1
_device = "cpu"
_print_every = 20

opt = torch.optim.SGD(model.parameters(), lr=lr, weight_decay = weight_decay, momentum = momentum)
# opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [243]:
# UWAGI:
# gradienty się liczą ale nie uaktualniają się wagi !!! 
# na razie nwm co z tym z robić mam inne zajęcia tez 

model = Model().to("cpu")
initialize_weights(model)

cnt +=  1
run_name = "run_nr " + str(cnt) + " larger simple nn, 100 games" 
# run = wandb.init(
#     project="Assigmnent4 task2",
#     name = run_name,
#     config={
#         "epochs": epochs,
#         "learning_rate": lr,
#         "optimizer": "SGD",
#         "momentum": momentum,
#         "weight_decay": weight_decay,
#         "batch & dropout": True,
#     },
# )
# wandb.watch(model, F.cross_entropy, log="all")

train(num_of_epochs=epochs, train_loader=data_loaders["train"], opt=opt, print_every=_print_every, device=_device)
# test_error_rate = compute_error_rate(model, data_loaders["test"], device=_device)
wandb.finish()

Epoch 1
policzone gradienty: 
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8410, 0.3698, 0.0000, 1.1425, 0.0000, 0.7396, 0.0000]])
tensor([0.0000, 0.3698])

parametry PRZED sieci: 
tensor([[ 2.6896e-01,  8.6646e-02,  2.5616e-01, -2.2037e-01,  2.3781e-02,
         -7.5069e-01, -4.2765e-01,  1.0346e+00],
        [-1.9008e-01, -1.2592e-04, -3.4191e-03,  5.7313e-01,  4.7272e-01,
         -4.1658e-01,  1.6169e-01,  5.2713e-01]], grad_fn=<CloneBackward0>)
True
tensor([0., 0.], grad_fn=<CloneBackward0>)
True

parametry PO sieci: 
tensor([[ 2.6896e-01,  8.6646e-02,  2.5616e-01, -2.2037e-01,  2.3781e-02,
         -7.5069e-01, -4.2765e-01,  1.0346e+00],
        [-1.9008e-01, -1.2592e-04, -3.4191e-03,  5.7313e-01,  4.7272e-01,
         -4.1658e-01,  1.6169e-01,  5.2713e-01]], grad_fn=<CloneBackward0>)
True
tensor([0., 0.], grad_fn=<CloneBackward0>)
True





In [209]:
loss = nn.MSELoss()
input = torch.randn(1, 8, requires_grad=True)
target = torch.randn(1, 8)
output = loss(input, target)
output.backward()

In [83]:
for p in model.parameters():
    p.requires_grad = False

model.eval()
with torch.no_grad():
    for x, y in data_loaders['train']:
        x = x.to("cpu")
        print(f"x = {x.shape}")
        # print(f"y = {y}")
        out = model(x)
        print(f"out = {out}")
        _, pred = out.max(dim=1)
        print(f"pred ={pred}")

        batch_err = (pred != y).sum().item() / out.size(0)

        print(batch_err)



x = torch.Size([128, 8])
out = tensor([[0.0613, 0.9387],
        [0.2143, 0.7857],
        [0.4949, 0.5051],
        [0.3931, 0.6069],
        [0.2866, 0.7134],
        [0.0061, 0.9939],
        [0.4897, 0.5103],
        [0.2143, 0.7857],
        [0.1170, 0.8830],
        [0.1189, 0.8811],
        [0.3309, 0.6691],
        [0.3431, 0.6569],
        [0.1569, 0.8431],
        [0.0269, 0.9731],
        [0.5503, 0.4497],
        [0.4624, 0.5376],
        [0.3431, 0.6569],
        [0.4624, 0.5376],
        [0.0796, 0.9204],
        [0.1535, 0.8465],
        [0.0401, 0.9599],
        [0.3309, 0.6691],
        [0.0796, 0.9204],
        [0.2356, 0.7644],
        [0.1535, 0.8465],
        [0.3431, 0.6569],
        [0.1056, 0.8944],
        [0.2143, 0.7857],
        [0.0858, 0.9142],
        [0.2143, 0.7857],
        [0.0779, 0.9221],
        [0.3309, 0.6691],
        [0.0245, 0.9755],
        [0.3309, 0.6691],
        [0.1717, 0.8283],
        [0.2371, 0.7629],
        [0.1965, 0.8035],
       